In [3]:
from typing import List
import torch
import torch._dynamo as dynamo

In [4]:
dynamo.config.suppress_errors = True

def my_compiler(gm: torch.fx.GraphModule, example_inputs: List[torch.tensor]):
    print("my_compiler() called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward  # return a python callable

@dynamo.optimize(my_compiler)
def foo(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b
foo(torch.randn(10), torch.randn(10))

my_compiler() called with FX graph:
opcode         name    target                                                  args         kwargs
-------------  ------  ------------------------------------------------------  -----------  --------
placeholder    l_a_    L_a_                                                    ()           {}
placeholder    l_b_    L_b_                                                    ()           {}
call_function  abs_1   <built-in method abs of type object at 0x7f66b9e81840>  (l_a_,)      {}
call_function  add     <built-in function add>                                 (abs_1, 1)   {}
call_function  x       <built-in function truediv>                             (l_a_, add)  {}
call_method    sum_1   sum                                                     (l_b_,)      {}
call_function  lt      <built-in function lt>                                  (sum_1, 0)   {}
output         output  output                                                  ((x, lt),)   {}
my_c

tensor([ 0.8042,  0.2048, -0.0870, -0.1531, -0.7538,  0.0711, -0.2518, -0.0065,
         0.4528,  0.0271])

In [5]:
print(torch._dynamo.list_backends())

['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'openxla_eval', 'tvm']
